Created by : Divye Joshi 

for any queries : divyej90_soe@jnu.ac.in

<h3>First we will load the required dependencies<h3>

In [1]:
import tensorflow as tf

from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict

import os

2023-09-26 11:48:22.145438: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-26 11:48:22.178339: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-26 11:48:22.178818: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-26 11:48:22.844387: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
_pipeline_root = "./pipeline/"
_data_root = "./data"
_data_filepath = os.path.join(_data_root, "dataset.csv")

In [3]:
context=InteractiveContext(pipeline_root=_pipeline_root)

<h3>The first step is to use Example gen to our .csv file. ExampleGen converts our csv data into (tf.data.TFRecordDataset) Format which is easy to tensorflow to read.<h3>

In [4]:
example_gen=tfx.components.CsvExampleGen(input_base=_data_root)

context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [7]:
context.show(example_gen.outputs['examples'])

<h3>This component has successfully run. You will be able to see the corresponding folders and directories inside our "pipeline" directory. This is called ml-metadata and contains the various generated components, such as our data which is stored by ExampleGen in the above location. The pipeline directory is also home to Schemas, Statistics, and even Model files.<h3>

<h3>Now, as I have told you. ExampleGen converts our data into (tf.data.TFRecordDataset) Format which is easy to tensorflow to read. That's why we cannot read it normally.<h3>

<h3>It is stored in the .gzip format and we need to create a function which can fetch n records from our data.<h3>


In [16]:
def get_records_gzip(filepath,num_records):
    dataset=tf.data.TFRecordDataset(filepath,compression_type="GZIP")
    records=[]
    for tfrecord in dataset.take(num_records):
            serialized_example=tfrecord.numpy()
            example=tf.train.Example()
            example.ParseFromString(serialized_example)
            example_dict=(MessageToDict(example))
            records.append(example_dict)
    return records


In [17]:
get_records_gzip("./pipeline/CsvExampleGen/examples/1/Split-train/data_tfrecord-00000-of-00001.gz",2)

[{'features': {'feature': {'hypertension': {'int64List': {'value': ['0']}},
    'heart_disease': {'int64List': {'value': ['1']}},
    'HbA1c_level': {'floatList': {'value': [6.6]}},
    'age': {'floatList': {'value': [80.0]}},
    'bmi': {'floatList': {'value': [25.19]}},
    'smoking_history': {'bytesList': {'value': ['bmV2ZXI=']}},
    'diabetes': {'int64List': {'value': ['0']}},
    'blood_glucose_level': {'int64List': {'value': ['140']}},
    'gender': {'bytesList': {'value': ['RmVtYWxl']}}}}},
 {'features': {'feature': {'hypertension': {'int64List': {'value': ['0']}},
    'diabetes': {'int64List': {'value': ['0']}},
    'smoking_history': {'bytesList': {'value': ['bmV2ZXI=']}},
    'gender': {'bytesList': {'value': ['TWFsZQ==']}},
    'bmi': {'floatList': {'value': [27.32]}},
    'age': {'floatList': {'value': [28.0]}},
    'blood_glucose_level': {'int64List': {'value': ['158']}},
    'heart_disease': {'int64List': {'value': ['0']}},
    'HbA1c_level': {'floatList': {'value': [5.7

<h3>As you can see, we can view our data using this function. It has been converted to a suitable (tf.data.TFRecordDataset) format.<h3>

<h3>The next step is to the StatisticsGen Component of TFx to get information about our data. This can help us find abnormalities (Example: The max. value of age is 1000)<h3>

In [5]:
statistics_gen=tfx.components.StatisticsGen(examples=example_gen.outputs["examples"])

context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [6]:
context.show(statistics_gen.outputs['statistics'])

<h3>Now we are going to use SchemaGen to create a Schema for our data<h3>

In [9]:
schema_gen=tfx.components.SchemaGen(statistics=statistics_gen.outputs['statistics'])
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [10]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'HbA1c_level',FLOAT,required,,-
'age',FLOAT,required,,-
'blood_glucose_level',INT,required,,-
'bmi',FLOAT,required,,-
'diabetes',INT,required,,-
'gender',STRING,required,,'gender'
'heart_disease',INT,required,,-
'hypertension',INT,required,,-
'smoking_history',STRING,required,,'smoking_history'


,Values
Domain,
'gender',"'Female', 'Male', 'Other'"
'smoking_history',"'No Info', 'current', 'ever', 'former', 'never', 'not current'"


<h3>We have created a preprocessing_function.py in this same directory. It contains a function which will apply different kinds of transformations to our data (OneHotEncoding,MinMaxScaling,Etc).<h3>


<h3>We will now use our data in (tf.data.TFRecordDataset) format, our generated schema, and the function to apply transformation to our data.<h3>

<h3>This will happen through the transform component of TFx (Tensorflow Extended)<h3>

In [18]:
tf.get_logger().setLevel("ERROR")

In [21]:
transform=tfx.components.Transform(examples=example_gen.outputs["examples"],schema=schema_gen.outputs["schema"],
                                   module_file=("preprocessing_function.py"))

context.run(transform)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying preprocessing_function.py -> build/lib
installing to /tmp/tmpdmzksj3s
running install
running install_lib
copying build/lib/preprocessing_function.py -> /tmp/tmpdmzksj3s
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmpdmzksj3s/tfx_user_code_Transform-0.0+2db7d7cc84de6943a108d35f5d6b8d5c63b99340a360eb33ebaa41ec6e386df8-py3.9.egg-info
running install_scripts
creating /tmp/tmpdmzksj3s/tfx_user_code_Transfo

/home/endymion/anaconda3/envs/core-ml/lib/python3.9/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


Processing ./pipeline/_wheels/tfx_user_code_Transform-0.0+2db7d7cc84de6943a108d35f5d6b8d5c63b99340a360eb33ebaa41ec6e386df8-py3-none-any.whl
Processing ./pipeline/_wheels/tfx_user_code_Transform-0.0+2db7d7cc84de6943a108d35f5d6b8d5c63b99340a360eb33ebaa41ec6e386df8-py3-none-any.whl
Processing ./pipeline/_wheels/tfx_user_code_Transform-0.0+2db7d7cc84de6943a108d35f5d6b8d5c63b99340a360eb33ebaa41ec6e386df8-py3-none-any.whl


ExecutionResult(
    component_id: Transform
    execution_id: 4
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

<h3>Now, we are going to get the location(URI) of our transformed data and view it using get_records_gzip<h3>

In [24]:
get_records_gzip("./pipeline/Transform/transformed_examples/4/Split-train/transformed_examples-00000-of-00001.gz",2)

[{'features': {'feature': {'Hypertension': {'int64List': {'value': ['0']}},
    'Smoking_status': {'floatList': {'value': [0.0,
       1.0,
       0.0,
       0.0,
       0.0,
       0.0,
       0.0]}},
    'Gender': {'floatList': {'value': [1.0, 0.0, 0.0, 0.0]}},
    'HbA1c_level': {'floatList': {'value': [0.56363636]}},
    'Blood_Glucose_Level': {'floatList': {'value': [0.27272728]}},
    'Age': {'floatList': {'value': [1.0]}},
    'Heart_Disease': {'int64List': {'value': ['1']}},
    'Diabetes': {'int64List': {'value': ['0']}},
    'BMI': {'floatList': {'value': [0.17717087]}}}}},
 {'features': {'feature': {'Smoking_status': {'floatList': {'value': [0.0,
       1.0,
       0.0,
       0.0,
       0.0,
       0.0,
       0.0]}},
    'Diabetes': {'int64List': {'value': ['0']}},
    'Heart_Disease': {'int64List': {'value': ['0']}},
    'Gender': {'floatList': {'value': [0.0, 1.0, 0.0, 0.0]}},
    'HbA1c_level': {'floatList': {'value': [0.39999998]}},
    'BMI': {'floatList': {'value':